In [30]:
import ast
import os
from urllib.parse import urlparse
import pprint
import pandas as pd
import json
from random import randint
import time
import math
import urllib.request

In [31]:
class Extractor:
    def __init__(self):
        urlList = []

    def readDataIntoList(self, filename):
        if filename == "SVM_C5.txt":
            
            with open(filename, "r") as fp:
                x = json.load(fp)
            return x
#         print("EXTR")
        with open(filename, "r") as fp:
            data = fp.read()
            x = ast.literal_eval(data)
            
        return x
    
    def jsondump(self, js, file):
        with open(file, 'w') as outfile:
            json.dump(js, outfile, indent=4)
        
    def fetchDomain(self, url):
        parsed_uri = urlparse(url)
        result = '{uri.scheme}://{uri.netloc}/'.format(uri=parsed_uri)
        #print(result)
        return result

    def clean(self, urls, minscore, maxscore, file):
        finalList = []

        for item in urls:
            d = dict()
            if item[1][1] == "NaN":
                continue
            d["url"] = item[1][0]
            d["score"] = item[1][1]
            d["domain"] = self.fetchDomain(item[1][0])
            finalList.append(d)
        # pprint.pprint(finalList, indent=4)
        uniqueDomains = dict()
        uniqueList = []
        for item in finalList:
            if item["score"]>=minscore and item["score"]<maxscore:
                if item["domain"] in uniqueDomains:
                    uniqueDomains[item["domain"]]["counts"] += 1
                    uniqueDomains[item["domain"]]["urls"].append([item["url"],item["score"]])
                else:
                    uniqueDomains[item["domain"]] = dict()
                    uniqueDomains[item["domain"]]["counts"] = 1
                    uniqueDomains[item["domain"]]["urls"] = [[item["url"],item["score"]]]
        # pprint.pprint(uniqueDomains)   
        print(len(uniqueDomains))
        if file!=-1:
            self.jsondump(uniqueDomains, "Final_Data/cleaned_data/"+file[:2]+"_top_all.json")    
        return uniqueDomains
    
    def extract(self, data, top):
        totalUnique = len(data) #total unique URLs
        finalList = []
        if top<totalUnique:
            for item in data:
                index = randint(0,len(data[item])-1)   
                finalList.append(data[item]["urls"][index][0]) 
        else:
            for item in data:
                i=0
                maxEach = int(top/totalUnique)
                while i<maxEach and i<len(data[item]["urls"]):
                    finalList.append(data[item]["urls"][i][0])
                    i+=1
            if len(finalList)<top:
                dif = top - len(finalList) # more urls needed
                i = 0
                for item in data:
                    if i>= dif:
                        break
                    count = 0
                    while True: 
                        index = randint(0,len(data[item]["urls"])-1)    
                        if data[item]["urls"][index] not in finalList:
                            finalList.append(data[item]["urls"][index][0])
                            i+=1
                            break
                        else:
                            if count>len(data[item]["urls"]): # exceeds 
                                break
                            count += 1
        return finalList  
    def cleanAllData(self, files_list, output_file):
        ex = Extractor()
        path = "Final_Data/"
        alldata=[]
        for f in files_list:
            print(f)
            if f[:2] == "SV":
                with open(path+f, "r") as fp:
                    data = json.load(fp)
            else:
                data = ex.readDataIntoList(path+f)
            print("data extracted")
            cleaned_data = ex.clean(data, -5, 5, -1)
            alldata.append(cleaned_data)
#             pprint.pprint(cleaned_data, indent=4)
        self.jsondump(alldata, output_file)
#             return alldata

In [32]:
# if __name__ == "__main__":
#     ex = Extractor()
#     path = "Final_Data/"
#     all_list = []
# #     extractCode = "All" # set to All for extracting all URLs with theur scores for each algorithm
#     if extractCode == "All":
#         filesList = []
#         for file in os.listdir(path):
#             if file[-5:] == "5.txt":
#                 filesList.append(file)
#         out=path+"CleanedDataAll.json"
#         ex.cleanAllData(filesList, out)
#     else:
#         for file in os.listdir(path):
#             if file[-5:] == "5.txt" or file[:6] == "Cosine": # 5 class urls
# #                 print(file)
#                 if file[:2] == "SV" or file[:6] == "Cosine":
#                     with open(path+file, "r") as fp:
#                         data = json.load(fp)
#                 else:
#                     data = ex.readDataIntoList(path+file)
#     #           file = "Final_Data/NB_C5.txt"
#                 top = 40 # total number of URLs to extract
#                 try:
#                     if file[:2] == "RF" or file[:2] == "NN":
#                         minscore = 3
#                         maxscore = 5
#                     elif file[:6] == "Cosine":
#                         minscore = 0.5
#                         maxscore = 1
#                     else:
#                         minscore = 4
#                         maxscore = 5
#                     print("--------------\nMax\n--------------\n")
#                     cleaned_data = ex.clean(data, minscore, maxscore, file)
#                     final_list = ex.extract(cleaned_data, top)
#     #                 pprint.pprint(final_list)
#                     all_list += final_list
#                 except:
#                     print("ERROR")
#                 try:
#                     print("--------------\nMin\n--------------\n")
#                     if file[:2] == "RF" or file[:2] == "NN":
#                         minscore = 2
#                         maxscore = 3
#                     elif file[:6] == "Cosine":
#                         minscore = 0.3
#                         maxscore = 0.5
#                     else:
#                         minscore = 3
#                         maxscore = 4
#                     cleaned_data = ex.clean(data, minscore, maxscore, file)
#                     final_list = ex.extract(cleaned_data, top)
#     #                 pprint.pprint(final_list)
#                     all_list += final_list
#                 except:
#                     print("ERROR")

#     #     pprint.pprint(all_list, indent=4)
#         print(len(all_list))
#         with open("Final_Data/final_list.txt", "a") as fp:
#             for item in all_list:
#                 fp.write(item+"\n")
    

In [33]:
ex = Extractor()
path = "Final_Data/"
all_list = dict()

for file in os.listdir(path):
#     print(file)
    data_list=[]
    try:
        if file[-5:] == "5.txt" or file == "Cosine_Results.txt": # 5 class urls
            print(file)
            if file[:2] == "SV" or file[:6] == "Cosine":
                with open(path+file, "r") as fp:
                    data = json.load(fp)
            else:
                data = ex.readDataIntoList(path+file)
            for item in data:
                
                data_dict=dict()
                
                if item[0][0] not in data_dict:
                    data_dict["url-id"] = item[0]
                    
                    # URL cleaning
                    url = item[1][0]
                    if "/xml/nlm" in url: # has XML text - same as corresponding e-publication
                        continue
                    if "article-abstract" in url: # remove abstract parts and keep the corresponding papers
                        continue
                    if "epub" in url: # remove e-publication download versions
                        continue
                    url = url.split("#")[0] # removing all parts after a # in a URL as they represent different parts of the same page
#                     print(url)
                    
                    if "http://" in url:
                        url = str(url[7:])
                    if "https://" in url:
                        url = str(url[8:])
#                     print(url)
                    url = url.rstrip("/") # removing trailing '/'
#                     print(url)
                    if "/full" == url[-5:]: # remove urls with full appendix
                        continue
                    if "/index.html" == url[-11:]:# remove urls with inde.html (url w/o it is collected)
                        continue
                    if "/call-for-papers/" in url:  # remove links with call for papers
                        continue
                    data_dict["url"] = url
                    data_dict["score"] = item[1][1]
                    
                    # extract domain from URL
                    url = url.split("/")[0]          
                    if ':' in url:
                        url = url.split(':')[0]
                        
                    if "#" in url:
                        url = url.split('#')[0]
                        
                    data_dict["domain"] = url # ex.fetchDomain(item[1][0])
        
                    data_list.append(data_dict)
            
            with open(path+"new_data/"+file[:2]+".json", "w") as fp:
                json.dump(data_list, fp, indent=4)

            print("Created file ",path,"new_data/",file[:2],".json")
    except Exception as e:
        print("ERROR: ", str(e), " File: ", file)    

RF_C5.txt
Created file  Final_Data/ new_data/ RF .json
NB_C5.txt
Created file  Final_Data/ new_data/ NB .json
NN_Fixed_C5.txt
Created file  Final_Data/ new_data/ NN .json
Cosine_Results.txt
Created file  Final_Data/ new_data/ Co .json
SVM_C5.txt
Created file  Final_Data/ new_data/ SV .json


In [34]:
path = "Final_Data/new_data/"
nested_data = dict()
for file in os.listdir(path):
    if file[2]!=".":
        continue
    print(file)
    
    with open(path+file, "r") as fp:
        data = json.load(fp)
        
    for item in data:
        if item["url"] not in nested_data:
            if ":" in item["domain"] or "#" in item["domain"]:
                print(item["domain"])
            nested_data[item["url"]] = dict()
            nested_data[item["url"]]["domain"] = item["domain"]
        nested_data[item["url"]][file[:2]] = item["score"]   

with open(path+"cumulative_data.json", "w") as fp:
    json.dump(nested_data, fp, indent=4)


RF.json
Co.json
SV.json
NN.json
NB.json


In [35]:
cleaned_nested = dict()
for item in nested_data:
#     print(item)
#     a = input()
    #excluding rows with NaNs
    if math.isnan(nested_data[item]['SV']) or math.isnan(nested_data[item]['NN']) or math.isnan(nested_data[item]['NB']) or math.isnan(nested_data[item]['RF']):
        continue
    #excluding rows with all 0 scores 
    if nested_data[item]['SV']==0 and nested_data[item]['NN']==0 and nested_data[item]['NB']==0 and nested_data[item]['RF']==0:
        continue
    #excluding rows with any negative scores 
    if nested_data[item]['SV']<0 or nested_data[item]['NN']<0 or nested_data[item]['NB']<0 or nested_data[item]['RF']<0:
        continue
    if item not in cleaned_nested:
        cleaned_nested[item] = nested_data[item]
#         cleaned_nested.append(item)

with open(path+"cumulative_cleaned_data.json", "w") as fp:
    json.dump(cleaned_nested, fp, indent=4)

In [36]:
df = pd.DataFrame(cleaned_nested)

In [37]:
df.head()

,academic.oup.com,academic.oup.com/DocumentLibrary/CDJ/FAQs.docx,academic.oup.com/DocumentLibrary/aobpla/main%20text%20template.doc,academic.oup.com/abm,academic.oup.com/abm/advance-article/doi/10.1093/abm/kay019/4984626,academic.oup.com/abm/advance-articles,academic.oup.com/advances,academic.oup.com/advances/CrossRef-CitedBy/4591560,academic.oup.com/advances/CrossRef-CitedBy/4591564,academic.oup.com/advances/advance-article/doi/10.1093/advances/nmy003/4964951,...,www.wmo.int/pages/prog/www/wigos/marine_pp.html,www.wmo.int/pages/prog/www/wigos/projects.html,www.wmo.int/pages/prog/www/wigos/technical_reports.html,www.wmo.int/pages/prog/www/wigos/tools.html,www.wmo.int/pages/rss/index_en.html,www.wmo.int/pages/rss/indexwidgetrss_en.html,www.wmo.int/pages/summary/cosponsored_summary_en.html,www.wmo.int/pages/summary/cosponsored_summary_fr.html,www.wmo.int/pages/summary/techcom_en.html,www.wmo.int/thorpex
Co,0.188982,0.308607,0.188982,0.188982,0.313993,0.192268,0.188982,0.254588,0.267261,0.339593,...,0.377292,0.308607,0.253546,0.188982,0.188982,0.188982,0.374654,0.188982,0.188982,0.253546
NB,0.495508,0.469466,0.725645,0.482731,1.28024,1.99996,0.482731,0.495538,0.768705,2.99968,...,1.00733,1.5998,0.584371,0.576443,0.576443,0.576443,1.85114,1,0.576443,0.579922
NN,2.0218,1.51977,2.39088,1.93824,1.783,2.75172,1.93824,1.7823,1.96139,2.61412,...,2.9617,2.00057,1.92744,1.95002,1.95002,1.95002,2.43007,0.992591,1.95002,2.26345
RF,0.284629,0.348374,0.452311,0.163139,1.33482,0.487993,0.163139,0.808915,0.335138,1.25071,...,1.09,0.797031,0.305527,0.206934,0.206934,0.206934,0.96,0.782572,0.206934,0.482432
SV,3,1,3,3,1,3,3,3,2.5,1,...,4,4,2.5,4,4,4,3,4,4,3


In [38]:
df=df.T

In [39]:
df.head(10)

,Co,NB,NN,RF,SV,domain
academic.oup.com,0.188982,0.495508,2.0218,0.284629,3,academic.oup.com
academic.oup.com/DocumentLibrary/CDJ/FAQs.docx,0.308607,0.469466,1.51977,0.348374,1,academic.oup.com
academic.oup.com/DocumentLibrary/aobpla/main%20text%20template.doc,0.188982,0.725645,2.39088,0.452311,3,academic.oup.com
academic.oup.com/abm,0.188982,0.482731,1.93824,0.163139,3,academic.oup.com
academic.oup.com/abm/advance-article/doi/10.1093/abm/kay019/4984626,0.313993,1.28024,1.783,1.33482,1,academic.oup.com
academic.oup.com/abm/advance-articles,0.192268,1.99996,2.75172,0.487993,3,academic.oup.com
academic.oup.com/advances,0.188982,0.482731,1.93824,0.163139,3,academic.oup.com
academic.oup.com/advances/CrossRef-CitedBy/4591560,0.254588,0.495538,1.7823,0.808915,3,academic.oup.com
academic.oup.com/advances/CrossRef-CitedBy/4591564,0.267261,0.768705,1.96139,0.335138,2.5,academic.oup.com
academic.oup.com/advances/advance-article/doi/10.1093/advances/nmy003/4964951,0.339593,2.99968,2.61412,1.25071,1,academic.oup.com


In [40]:
test = df.sort_values(['RF','NN','NB','SV','Co'], ascending=False)

In [41]:
test.head(20)

,Co,NB,NN,RF,SV,domain
www.frontiersin.org/articles/10.3389/fmars.2017.00059,0.392137,4,3.95835,3.88,4,www.frontiersin.org
www.mdpi.com/rss/journal/geosciences,0.613927,4,1.92005,3.44,1,www.mdpi.com
academic.oup.com/icesjms/article/70/6/1220/635452,0.355276,1.08799,1.00326,3.41,2.5,academic.oup.com
academic.oup.com/icesjms/article/71/4/867/666460,0.349715,4,2.0001,3.38,1,academic.oup.com
www.wmo.int/pages/prog/amp/mmop/documents/dbcp/Dbcp30/Abstracts.html,0.431082,2.99997,2.67842,3.37,3,www.wmo.int
academic.oup.com/icesjms/article/72/3/1057/690224,0.43314,4,3.00089,3.27,4,academic.oup.com
www.frontiersin.org/articles/10.3389/fmars.2018.00066,0.414796,4,2.91164,3.25,2.5,www.frontiersin.org
www.frontiersin.org/articles/10.3389/fmars.2017.00277,0.463789,2.72089,2.41832,3.22,2.5,www.frontiersin.org
academic.oup.com/icesjms/article/72/5/1399/765328,0.358901,1.00002,3.77343,3.21,4,academic.oup.com
www.frontiersin.org/articles/10.3389/fmars.2017.00295,0.351657,2.40173,3.22214,3.2,3,www.frontiersin.org


In [42]:
test.shape

(30732, 6)

In [43]:
domains = test.domain.unique()

In [44]:
cleaned_top = pd.DataFrame()
for d in domains:
    cleaned_top = cleaned_top.append(test[test["domain"] == d])

In [45]:
cleaned_top = cleaned_top.sort_values(['RF','NN','NB','SV','Co'], ascending=False)

In [46]:
cleaned_top.to_csv("Final_Data/new_data/cleaned_top_all_new.csv")

In [47]:
cleaned_top.domain.unique()

array(['www.frontiersin.org', 'www.mdpi.com', 'academic.oup.com',
       'www.wmo.int', 'www.jamstec.go.jp', 'eurogoos.eu',
       'link.springer.com', 'www.fao.org', 'www.data.gov',
       'www.omicsonline.org', 'www.journals.elsevier.com'], dtype=object)

In [48]:
nn_urls = pd.read_csv("Final_Data/cleaned_data/nn_urls.csv")

In [49]:
nn_urls.head()

,Urls,Co,NB,NN,RF,SV,domain
0,www.frontiersin.org/articles/10.3389/fmars.201...,0.392137,4.000000,3.958352,3.88,4.0,www.frontiersin.org
1,www.mdpi.com/rss/journal/geosciences,0.613927,4.000000,1.920046,3.44,1.0,www.mdpi.com
2,academic.oup.com/icesjms/article/70/6/1220/635452,0.355276,1.087993,1.003265,3.41,2.5,academic.oup.com
3,academic.oup.com/icesjms/article/71/4/867/666460,0.349715,4.000000,2.000097,3.38,1.0,academic.oup.com
4,www.wmo.int/pages/prog/amp/mmop/documents/dbcp...,0.431082,2.999966,2.678422,3.37,3.0,www.wmo.int


In [52]:

urls = nn_urls["Urls"]
valid = []
for url in urls:
    print(len(valid))
    try:
#         request = requests.get("http://"+url)
        code = urllib.request.urlopen("http://"+url).getcode()
        if code == 200:
            valid.append("Active")
        else:
            valid.append("Inactive")
    except Exception as e:
        print(str(e))
        valid.append("Active")
print(len(valid))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
<urlopen error [Errno 8] nodename nor servname provided, or not known>
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
<urlopen error [Errno 8] nodename nor servname provided, or not known>
227
228
229
230
231
232
233
234
HTTP Error 403: Forbidden
23

In [53]:
nn_urls["validity"] = pd.Series(valid).values

In [54]:
nn_urls.head()

,Urls,Co,NB,NN,RF,SV,domain,validity
0,www.frontiersin.org/articles/10.3389/fmars.201...,0.392137,4.000000,3.958352,3.88,4.0,www.frontiersin.org,Active
1,www.mdpi.com/rss/journal/geosciences,0.613927,4.000000,1.920046,3.44,1.0,www.mdpi.com,Active
2,academic.oup.com/icesjms/article/70/6/1220/635452,0.355276,1.087993,1.003265,3.41,2.5,academic.oup.com,Active
3,academic.oup.com/icesjms/article/71/4/867/666460,0.349715,4.000000,2.000097,3.38,1.0,academic.oup.com,Active
4,www.wmo.int/pages/prog/amp/mmop/documents/dbcp...,0.431082,2.999966,2.678422,3.37,3.0,www.wmo.int,Active


In [55]:
nn_urls.to_csv("NN_urls.csv")